<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h1 style='margin:10px 5px'> 
Master Thesis Yannik Haller - Regression Analysis
</h1>
</div>

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
Preparation: load packages, set the appropriate working directory and load the data
</h2>
</div>

In [1]:
# Import required baseline packages
import re
import os
import glob
import time
import calendar
import sys
import pandas as pd
import numpy as np

# Change pandas' setting to print out long strings
pd.options.display.max_colwidth = 200

# Plotting tools
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# Set global parameters for plotting
import matplotlib.pylab as pylab
params = {'legend.fontsize': 10,
          'figure.figsize': (8, 6),
          'axes.labelsize': 14,
          'axes.titlesize': 16,
          'xtick.labelsize': 10,
          'ytick.labelsize': 10}
pylab.rcParams.update(params)

# Regression and smoothing tools
import statsmodels.api as sm
import statsmodels.stats.diagnostic as ssd
from stargazer.stargazer import Stargazer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold

# Disable warnings
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)
warnings.filterwarnings("ignore", category = FutureWarning)

In [2]:
# Set the appropriate working directory
os.chdir('D:\\Dropbox\\MA_data')

In [3]:
# Read in the sparse data as follows
articles = pd.read_csv("Analysis/articles_main_sparse.csv", index_col = 0)
# Transform publication_date into a daily date type
articles.publication_date = pd.to_datetime(articles.publication_date, yearfirst = True).astype('datetime64[D]')
# Transform the Topic_ID_2 into an integer type (not possible to read in as integer, due to missing values)
articles['Topic_ID_2'] = articles['Topic_ID_2'].astype("Int32")
# Take a look at the dataframe
articles

,source_short,source_long,publication_date,language,year,month,week,weekday,year_month,year_week,...,Topic_fine,Vader_polarity,Vader_polarity_adj,Vader_polarity_adj_2,Blob_polarity,weighted_stridx,weighted_dconfirmed_pm,weighted_ddeaths_pm,weighted_7dconfirmed_pm,weighted_7ddeaths_pm
0,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,politics_international,0.9785,0.1582,0.1582,0.124348,40.256409,146.671235,1.706711,92.683854,0.487632
1,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,economy_international,-0.7772,-0.1196,-0.1996,0.100000,40.256409,146.671235,1.706711,92.683854,0.487632
2,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,economy_international,0.7561,0.0414,-0.0218,0.163700,40.256409,146.671235,1.706711,92.683854,0.487632
3,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,economy_international,0.8078,0.0780,0.0780,-0.012364,40.256409,146.671235,1.706711,92.683854,0.487632
4,AGE,Agefi,2020-10-01,fr,2020,10,39,Thursday,2020-10,2020-39,...,economy_international,-0.8205,-0.0869,-0.1215,0.207273,40.256409,146.671235,1.706711,92.683854,0.487632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441178,ZWA,20 minuten,2021-01-22,de,2021,1,3,Friday,2021-01,2021-03,...,economy_national,0.9693,0.1817,0.1817,-0.100000,75.000000,245.907497,5.659186,228.809826,6.222238
2441179,ZWA,20 minuten,2021-01-22,de,2021,1,3,Friday,2021-01,2021-03,...,inconsequential,-0.7441,-0.0514,-0.0514,0.121429,75.000000,245.907497,5.659186,228.809826,6.222238
2441180,ZWA,20 minuten,2021-01-22,de,2021,1,3,Friday,2021-01,2021-03,...,sports,0.8391,0.2133,0.1293,0.350000,75.000000,245.907497,5.659186,228.809826,6.222238
2441181,ZWA,20 minuten,2021-01-22,de,2021,1,3,Friday,2021-01,2021-03,...,tragedies_crimes,-0.8689,-0.5667,-0.5667,-0.433333,75.000000,245.907497,5.659186,228.809826,6.222238


In [4]:
# Transform the Polarity scores Appropriately
### After appropriately combining the language-specific polarity scores of the VPA, VPAII, and BP into a single variable each, 
### rescale these sentiment measures to have a standard deviation of 1 (such that they are comparable to each other) and store them into the DF
## VPA
# Rescale the polarity scores and store the results in the Vader_polarity_adj_rsc variable
articles['Vader_polarity_adj'] = articles['Vader_polarity_adj'].copy()/np.std(articles['Vader_polarity_adj'])
## VPAII
# Rescale the polarity scores and store the results in the Vader_polarity_adj_2_rsc variable
articles['Vader_polarity_adj_2'] = articles['Vader_polarity_adj_2'].copy()/np.std(articles['Vader_polarity_adj_2'])
## BP
# Rescale the language-specific BPs prior to merging them
articles.loc[articles['language'] == 'de', 'Blob_polarity'] = (articles.loc[articles['language'] == 'de', 'Blob_polarity'].copy()/
                                                                                np.std(articles.loc[articles['language'] == 'de', 'Blob_polarity']))
articles.loc[articles['language'] == 'fr', 'Blob_polarity'] = (articles.loc[articles['language'] == 'fr', 'Blob_polarity'].copy()/
                                                                                np.std(articles.loc[articles['language'] == 'fr', 'Blob_polarity']))
articles.loc[articles['language'] == 'it', 'Blob_polarity'] = (articles.loc[articles['language'] == 'it', 'Blob_polarity'].copy()/
                                                                                np.std(articles.loc[articles['language'] == 'it', 'Blob_polarity']))
# Rescale the combined polarity scores again and store the results in the Blob_polarity_rsc variable
articles['Blob_polarity'] = articles['Blob_polarity'].copy()/np.std(articles['Blob_polarity'])

In [5]:
# Take a look at the size of the prepared data
sys.getsizeof(articles)

1845667922

In [6]:
# Take a look at some information concerning the articles dataframe
articles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2439096 entries, 0 to 2441182
Data columns (total 28 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   source_short             object        
 1   source_long              object        
 2   publication_date         datetime64[ns]
 3   language                 object        
 4   year                     int64         
 5   month                    int64         
 6   week                     int64         
 7   weekday                  object        
 8   year_month               object        
 9   year_week                object        
 10  Topic_ID_1               int64         
 11  Affiliation_Prob_1       float64       
 12  Topic_ID_2               Int32         
 13  Affiliation_Prob_2       float64       
 14  Topic_1                  object        
 15  Topic_2                  object        
 16  Topic_ID_fine            float64       
 17  Affiliation_Prob_fine    fl

In [7]:
# Get the abbreviation of all newspapers that publish articles exclusively in German
de_src = []
for source in np.unique(articles['source_short']):
    n_lang = np.unique(articles.loc[articles['source_short'] == source, 'language'])
    if len(n_lang) == 1 and n_lang[0] == 'de':
        de_src.append(source)

# Get the abbreviation of all newspapers that publish articles exclusively in French
fr_src = []
for source in np.unique(articles['source_short']):
    n_lang = np.unique(articles.loc[articles['source_short'] == source, 'language'])
    if len(n_lang) == 1 and n_lang[0] == 'fr':
        fr_src.append(source)

# Get the abbreviation of all newspapers that publish articles exclusively in Italian
it_src = []
for source in np.unique(articles['source_short']):
    n_lang = np.unique(articles.loc[articles['source_short'] == source, 'language'])
    if len(n_lang) == 1 and n_lang[0] == 'it':
        it_src.append(source)

In [8]:
# Take a look newspapers that publish articles in multiple languages
for source in np.unique(articles['source_short']):
    n_lang = np.unique(articles.loc[articles['source_short'] == source, 'language'], return_counts = True)
    if len(n_lang[0]) > 1:
        total_count = n_lang[1].sum()
        if len(n_lang[0]) == 2:
            string_to_print = n_lang[0][0]+' ('+str(np.round(n_lang[1][0]*100/total_count,3))+'%),\t'+n_lang[0][1]+' ('+\
                                                str(np.round(n_lang[1][1]*100/total_count,3))+'%)'
        if len(n_lang[0]) == 3:
            string_to_print = n_lang[0][0]+' ('+str(np.round(n_lang[1][0]*100/total_count,3))+'%),\t'+n_lang[0][1]+' ('+\
                                                str(np.round(n_lang[1][1]*100/total_count,3))+'%),\t'+n_lang[0][2]+' ('+\
                                                str(np.round(n_lang[1][2]*100/total_count,3))+'%)'
        print(source+' publishes articles in the following languages:\t'.expandtabs(55-len(source)) + string_to_print)

AZM publishes articles in the following languages:     de (99.989%),	fr (0.011%)
BAZ publishes articles in the following languages:     de (99.997%),	fr (0.003%)
BEOL publishes articles in the following languages:    de (99.994%),	fr (0.006%)
BIT publishes articles in the following languages:     de (99.954%),	fr (0.046%)
BLI publishes articles in the following languages:     de (99.996%),	fr (0.004%)
BT publishes articles in the following languages:      de (99.973%),	fr (0.027%)
BUET publishes articles in the following languages:    de (99.996%),	it (0.004%)
CASO publishes articles in the following languages:    de (99.999%),	fr (0.001%)
FN publishes articles in the following languages:      de (99.966%),	fr (0.034%)
INFS publishes articles in the following languages:    de (99.818%),	fr (0.182%)
LTZ publishes articles in the following languages:     de (99.987%),	fr (0.013%)
NNBS publishes articles in the following languages:    de (99.998%),	fr (0.002%)
NNHEU publishes articles in 

We observe that 78 out of the 96 distinct sources exclusively publish articles written in one language. Furthermore, for 17 out of the 18 sources that release articles in multiple languages, at least 98.5% of the publications are also written in the same language. Thus, we suspect that including fixed effects for the different languages and the different sources at the same time causes severe multicollinearity issues, since for each language one could construct a linear combination with the source inidcating dummies that highly correlates with the respective language indicating dummy. To prevent this issue we could either exclude language fixed effects or exclude source fixed effects. While the latter strategy would lead to conveniently interpretable results, the former would deliver more detailed (and thus more valuable) results. However, since the present analysis aims at revealing the difference between the reactions of the distinct sources, we decide to exclude language fixed effects in the following. However, as most sources tend to publish articles in only one langage, we decide to indicate the prevailing language of each source at the beginning of each source's abbreviation.

In [9]:
# Add the newspapers that publish predominantly German articles to the list of newspapers that publish exclusively in German
de_to_add = ['AZM', 'BAZ', 'BEOL', 'BIT', 'BLI', 'BT', 'BUET', 'CASO', 'FN', 'INFS', 'LTZ', 'NNBS', 'OLT', 'SOS', 'SOZM']
[de_src.append(src) for src in de_to_add]
# Add the newspapers that publish predominantly French articles to the list of newspapers that publish exclusively in French
fr_to_add = ['NNHEU', 'NNTDG']
[fr_src.append(src) for src in fr_to_add]
print('') # suppress output

In [10]:
# Indicate the according language for each source
for source in de_src:
    articles.loc[articles['source_short'] == source, 'source_short'] = 'de_'+source
for source in fr_src:
    articles.loc[articles['source_short'] == source, 'source_short'] = 'fr_'+source
for source in it_src:
    articles.loc[articles['source_short'] == source, 'source_short'] = 'it_'+source
articles.loc[articles['source_short'] == 'SWII', 'source_short'] = 'defrit_SWII'

In [11]:
# Take a look at the adjusted source abbreviations
np.unique(articles['source_short'])

array(['de_APPZ', 'de_AVU', 'de_AZM', 'de_BAZ', 'de_BEOL', 'de_BIT',
       'de_BIZO', 'de_BLI', 'de_BODU', 'de_BT', 'de_BU', 'de_BUET',
       'de_BZ', 'de_BZM', 'de_CASO', 'de_COOP', 'de_FN', 'de_FURT',
       'de_FUW', 'de_FUWO', 'de_GLAT', 'de_GP', 'de_INFS', 'de_LB',
       'de_LTZ', 'de_LUZ', 'de_MEWO', 'de_MM', 'de_NIW', 'de_NLZS',
       'de_NNBE', 'de_NNBS', 'de_NNBU', 'de_NNTA', 'de_NZZ', 'de_NZZS',
       'de_OAS', 'de_OBW', 'de_OLT', 'de_ONA', 'de_RUEM', 'de_SBAU',
       'de_SBLI', 'de_SEBO', 'de_SF', 'de_SGT', 'de_SHZ', 'de_SHZO',
       'de_SI', 'de_SOS', 'de_SOZM', 'de_SRF', 'de_TA', 'de_TAGZ',
       'de_TAM', 'de_TAS', 'de_TBT', 'de_TZ', 'de_URZ', 'de_VOLK',
       'de_WASO', 'de_WB', 'de_WEOB', 'de_WEW', 'de_WILB', 'de_WOZ',
       'de_ZHOL', 'de_ZHUL', 'de_ZOF', 'de_ZPLU', 'de_ZSZ', 'de_ZUGZ',
       'de_ZWA', 'de_ZWAO', 'defrit_SWII', 'fr_AGE', 'fr_ARC', 'fr_COOF',
       'fr_GHI', 'fr_HEU', 'fr_ILLE', 'fr_JJ', 'fr_LBH', 'fr_LIB',
       'fr_MME', 'fr_NNHEU', 'fr_N

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
2. Robustness checks (RC2 & RC3)
</h2>
</div>

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h3 style='margin:10px 5px'> 
2.1 Comparison to models that use adjusted Vader polarity relying on the original sentiment lexica and TextBlob polarity as the label (RC2)
</h3>
</div>

In [12]:
# Remove columns that are not relevant for the regression analysis
articles_01 = articles.drop(['source_long','language','year','month','week','weekday','year_month','year_week','Topic_ID_1',
                             'Affiliation_Prob_1','Topic_ID_2','Affiliation_Prob_2','Topic_1','Topic_2',
                             'Topic_ID_fine','Affiliation_Prob_fine','Vader_polarity'], axis = 1).copy()

In [13]:
# In order to economize computation of the subsequent models, we exploit the fact that the weighted Stringency index and weighted Covid metrics do not vary for a source within the same day
## Hence we reduce the dataframe to each source-day-topic combination, while retrieving the corresponding average polarity scores (VPA, VPAII, and BP) and frequencies. That is, for each source and each topic, 
## we count the number of articles the considered source published on each day on the focal topic, and compute the average polarity score among these articles.
## The number of articles is then used to weight each observation in the subsequent regression models
articles_01 = articles_01.groupby(['source_short','Topic_fine','publication_date'])['Vader_polarity_adj', 'Vader_polarity_adj_2', 'Blob_polarity', 'weighted_stridx',
                                                                                    'weighted_dconfirmed_pm', 'weighted_ddeaths_pm','weighted_7dconfirmed_pm', 'weighted_7ddeaths_pm'].mean().reset_index().merge(
    articles_01.groupby(['source_short','Topic_fine','publication_date'])['Vader_polarity_adj'].count().reset_index().rename(columns = {'Vader_polarity_adj': 'count'}), 
    how = 'inner', left_on = ['source_short','Topic_fine','publication_date'], right_on = ['source_short','Topic_fine','publication_date'], validate = "1:1")

# Take a look at the created dataframe
articles_01

,source_short,Topic_fine,publication_date,Vader_polarity_adj,Vader_polarity_adj_2,Blob_polarity,weighted_stridx,weighted_dconfirmed_pm,weighted_ddeaths_pm,weighted_7dconfirmed_pm,weighted_7ddeaths_pm,count
0,de_APPZ,COVID_economy,2019-02-02,0.689970,0.641458,-0.161747,0.000000,0.000000,0.000000,0.000000,0.000000,1
1,de_APPZ,COVID_economy,2019-02-06,-0.446321,-0.414940,2.062278,0.000000,0.000000,0.000000,0.000000,0.000000,1
2,de_APPZ,COVID_economy,2019-03-20,-0.110750,-0.102963,0.363931,0.000000,0.000000,0.000000,0.000000,0.000000,1
3,de_APPZ,COVID_economy,2019-04-13,-0.761403,-0.707869,-2.426209,0.000000,0.000000,0.000000,0.000000,0.000000,1
4,de_APPZ,COVID_economy,2019-05-09,1.721049,1.600042,-0.424587,0.000000,0.000000,0.000000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
512308,it_ZWAI,transportation,2020-12-16,1.959519,1.821745,1.741135,61.666668,915.332160,40.104162,767.403266,26.641639,2
512309,it_ZWAI,transportation,2020-12-23,1.282541,1.117131,1.139605,72.469512,809.484930,11.485021,713.196048,16.806232,3
512310,it_ZWAI,transportation,2020-12-28,1.095387,0.794018,0.973308,72.469512,465.494770,22.907768,661.744109,19.209482,1
512311,it_ZWAI,transportation,2020-12-30,1.833062,1.704180,1.628772,72.469512,891.054364,25.701180,680.891317,22.065168,2


In [14]:
# Rename the source 'NZZ' such that it is chosen as a reference class later on
articles_01['source_short'].replace(['de_NZZ'], ['0_de_NZZ'], inplace = True)
# Rename the topic tragedies & crimes such that it is chosen as a reference class later on
## Note: we choose this topic as a reference class due to two reasons: 
### (1) the average polarity score is comparably low (such that FE coefficients of most other topics should be positivie) 
### (2) the curve depicting the development of the topic's average polarity over time reveals that the polarity is comparably rather stable and not very strongly affected by changes in the stringency index
articles_01['Topic_fine'].replace(['tragedies_crimes'], ['0_tragedies_crimes'], inplace = True)

# Create a dataframe which suits the structure required in the regression analysis
## Add the numerical variables
reg_df = articles_01[articles_01.columns[3:]].copy()

## Add the categorical variables
### Note: We use one hot encoding to transform each categorical feature into an indicator variable for each category. To prevent multicollinearity issues, 
### we drop the first category of each discrete variable as a reference class

# Publication Date
reg_df = reg_df.merge(pd.get_dummies(articles_01['publication_date'], drop_first = True, prefix = 'date'),
                                how = 'inner', left_index = True, right_index = True, validate = "1:1") # Reference class: 2019-01-01
# Topic_fine
reg_df = reg_df.merge(pd.get_dummies(articles_01['Topic_fine'], drop_first = True, prefix = 'topic'), 
                                how = 'inner', left_index = True, right_index = True, validate = "1:1") # Reference class: tragedies & crimes
# Source
reg_df = reg_df.merge(pd.get_dummies(articles_01['source_short'], drop_first = True, prefix = 'src'),
                                how = 'inner', left_index = True, right_index = True, validate = "1:1") # Reference class: de_NZZ


# Remove the articles_01 dataframe to save memory
del articles_01

# Transform the contained covid metrics from cases/deaths per million population to cases/deaths per 10K population
reg_df[reg_df.columns[4:8]] = reg_df[reg_df.columns[4:8]].copy() / 100
# Rename the modified columns accordingly
reg_df.rename(columns = {'weighted_dconfirmed_pm': 'weighted_dcases_p10k', 'weighted_ddeaths_pm': 'weighted_ddeaths_p10k', 
                         'weighted_7dconfirmed_pm': 'weighted_7dcases_p10k', 'weighted_7ddeaths_pm': 'weighted_7ddeaths_p10k', 'weighted_stridx': 'weighted_KOFSPI'}, inplace = True)

# Take a look at the created dataframe
reg_df

,Vader_polarity_adj,Vader_polarity_adj_2,Blob_polarity,weighted_KOFSPI,weighted_dcases_p10k,weighted_ddeaths_p10k,weighted_7dcases_p10k,weighted_7ddeaths_p10k,count,date_2019-01-02 00:00:00,...,src_fr_NNTDG,src_fr_NNTLM,src_fr_NOU,src_fr_RTS,src_fr_TDG,src_fr_TLMD,src_fr_TPS,src_fr_ZWAS,src_it_COOI,src_it_ZWAI
0,0.689970,0.641458,-0.161747,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.446321,-0.414940,2.062278,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.110750,-0.102963,0.363931,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.761403,-0.707869,-2.426209,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1.721049,1.600042,-0.424587,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512308,1.959519,1.821745,1.741135,61.666668,9.153322,0.401042,7.674033,0.266416,2,0,...,0,0,0,0,0,0,0,0,0,1
512309,1.282541,1.117131,1.139605,72.469512,8.094849,0.114850,7.131960,0.168062,3,0,...,0,0,0,0,0,0,0,0,0,1
512310,1.095387,0.794018,0.973308,72.469512,4.654948,0.229078,6.617441,0.192095,1,0,...,0,0,0,0,0,0,0,0,0,1
512311,1.833062,1.704180,1.628772,72.469512,8.910544,0.257012,6.808913,0.220652,2,0,...,0,0,0,0,0,0,0,0,0,1


In [15]:
## Get the feature variables to use in the regression
# Variables of interest
features = reg_df[['weighted_KOFSPI', 'weighted_dcases_p10k']].copy()
# Control variables
features = features.merge(reg_df[reg_df.columns[9:]], how = 'inner', left_index = True, right_index = True, validate = "1:1")
# Add a constant
features = sm.add_constant(features)

# Get the target variable / label
label_1 = reg_df[['Vader_polarity_adj_2']].copy()
label_2 = reg_df[['Vader_polarity_adj']].copy()
label_3 = reg_df[['Blob_polarity']].copy()

In [16]:
## Create the different feature selection sets to use subsequently
# Basic model without any fixed effects
selection_1 = [0,1,2]
# Add time fixed effects
selection_2 = selection_1.copy()
[selection_2.append(num) for num in np.arange(3,743)] # days
# Add cross-sectional fixed effects
selection_3 = selection_2.copy()
[selection_3.append(num) for num in np.arange(743,770)] # topics
[selection_3.append(num) for num in np.arange(770,865)] # sources

# Estimate a first set of models using the variable of interest together with cross sectional and time fixed effects (FE)
model_01 = sm.WLS(label_1, features[features.columns[selection_3]], weights = reg_df['count']).fit(cov_type = 'HC3')
model_02 = sm.WLS(label_2, features[features.columns[selection_3]], weights = reg_df['count']).fit(cov_type = 'HC3')
model_03 = sm.WLS(label_3, features[features.columns[selection_3]], weights = reg_df['count']).fit(cov_type = 'HC3')

In [17]:
# Take a look at the results of the models without the interaction variables
stargazer = Stargazer([model_01, model_02, model_03])
stargazer.title('Effect of the KOFSPI & Daily Confirmed Covid Cases per 10K Population (Models with CS & T FE) on VPAII, VPA, & BP')
stargazer.custom_columns(['Label: Adjusted Vader Polarity 2 (RM1C)', 'Label: Adjusted Vader Polarity (RC2A)', 'Label: TextBlob Polarity (RC2B)'], [1, 1, 1])
stargazer.show_model_numbers(False)
stargazer.significant_digits(5)
coef_ordered = features.columns[selection_1].tolist()
stargazer.covariate_order(coef_ordered)
stargazer
#stargazer.render_latex()

# Note: Reference classes
## Topic_fine: tragedies & crimes
## Source:     de_NZZ
## Day:        2019-01-01

In [18]:
# Remove the reg_df dataframe to save memory
del reg_df

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h3 style='margin:10px 5px'> 
2.2 Remove articles for which the readership data (and therefore appropriate cantonal weighting) is not available (RC3)
</h3>
</div>

In [19]:
# Remove columns that are not relevant for the regression analysis
articles_02 = articles.drop(['source_long','language','year','month','week','weekday','year_month','year_week','Topic_ID_1',
                             'Affiliation_Prob_1','Topic_ID_2','Affiliation_Prob_2','Topic_1','Topic_2',
                             'Topic_ID_fine','Affiliation_Prob_fine','Vader_polarity'], axis = 1).copy()
## Define the set of news outlets for which the readership data is not available
sources_wo_readership_data = ['de_AVU', 'fr_ARC', 'de_BT', 'de_BEOL', 'de_BUET', 'de_SHZO', 'de_INFS', 'fr_LBH', 'de_MEWO', 'de_RUEM', 
                              'de_SBAU', 'de_SEBO', 'de_TBT', 'de_VOLK', 'fr_RTS', 'de_SRF', 'defrit_SWII', 'de_WASO', 'de_ZPLU']
## Remove articles for which the readership data is not available
articles_02 = articles_02.loc[np.logical_not(articles_02.source_short.isin(sources_wo_readership_data))]

In [20]:
# In order to economize computation of the subsequent models, we exploit the fact that the weighted Stringency index and weighted Covid metrics do not vary for a source within the same day
## Hence we reduce the dataframe to each source-day-topic combination, while retrieving the corresponding average polarity scores (VPA, VPAII, and BP) and frequencies. That is, for each source and each topic, 
## we count the number of articles the considered source published on each day on the focal topic, and compute the average polarity score among these articles.
## The number of articles is then used to weight each observation in the subsequent regression models
articles_02 = articles_02.groupby(['source_short','Topic_fine','publication_date'])['Vader_polarity_adj', 'Vader_polarity_adj_2', 'Blob_polarity', 'weighted_stridx',
                                                                                    'weighted_dconfirmed_pm', 'weighted_ddeaths_pm','weighted_7dconfirmed_pm', 'weighted_7ddeaths_pm'].mean().reset_index().merge(
    articles_02.groupby(['source_short','Topic_fine','publication_date'])['Vader_polarity_adj'].count().reset_index().rename(columns = {'Vader_polarity_adj': 'count'}), 
    how = 'inner', left_on = ['source_short','Topic_fine','publication_date'], right_on = ['source_short','Topic_fine','publication_date'], validate = "1:1")

# Take a look at the created dataframe
articles_02

,source_short,Topic_fine,publication_date,Vader_polarity_adj,Vader_polarity_adj_2,Blob_polarity,weighted_stridx,weighted_dconfirmed_pm,weighted_ddeaths_pm,weighted_7dconfirmed_pm,weighted_7ddeaths_pm,count
0,de_APPZ,COVID_economy,2019-02-02,0.689970,0.641458,-0.161747,0.000000,0.000000,0.000000,0.000000,0.000000,1
1,de_APPZ,COVID_economy,2019-02-06,-0.446321,-0.414940,2.062278,0.000000,0.000000,0.000000,0.000000,0.000000,1
2,de_APPZ,COVID_economy,2019-03-20,-0.110750,-0.102963,0.363931,0.000000,0.000000,0.000000,0.000000,0.000000,1
3,de_APPZ,COVID_economy,2019-04-13,-0.761403,-0.707869,-2.426209,0.000000,0.000000,0.000000,0.000000,0.000000,1
4,de_APPZ,COVID_economy,2019-05-09,1.721049,1.600042,-0.424587,0.000000,0.000000,0.000000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
426408,it_ZWAI,transportation,2020-12-16,1.959519,1.821745,1.741135,61.666668,915.332160,40.104162,767.403266,26.641639,2
426409,it_ZWAI,transportation,2020-12-23,1.282541,1.117131,1.139605,72.469512,809.484930,11.485021,713.196048,16.806232,3
426410,it_ZWAI,transportation,2020-12-28,1.095387,0.794018,0.973308,72.469512,465.494770,22.907768,661.744109,19.209482,1
426411,it_ZWAI,transportation,2020-12-30,1.833062,1.704180,1.628772,72.469512,891.054364,25.701180,680.891317,22.065168,2


In [21]:
# Rename the source 'NZZ' such that it is chosen as a reference class later on
articles_02['source_short'].replace(['de_NZZ'], ['0_de_NZZ'], inplace = True)
# Rename the topic tragedies & crimes such that it is chosen as a reference class later on
## Note: we choose this topic as a reference class due to two reasons: 
### (1) the average polarity score is comparably low (such that FE coefficients of most other topics should be positivie) 
### (2) the curve depicting the development of the topic's average polarity over time reveals that the polarity is comparably rather stable and not very strongly affected by changes in the stringency index
articles_02['Topic_fine'].replace(['tragedies_crimes'], ['0_tragedies_crimes'], inplace = True)

# Create a dataframe which suits the structure required in the regression analysis
## Add the numerical variables
reg_df = articles_02[articles_02.columns[3:]].copy()

## Add the categorical variables
### Note: We use one hot encoding to transform each categorical feature into an indicator variable for each category. To prevent multicollinearity issues, 
### we drop the first category of each discrete variable as a reference class

# Publication Date
reg_df = reg_df.merge(pd.get_dummies(articles_02['publication_date'], drop_first = True, prefix = 'date'),
                                how = 'inner', left_index = True, right_index = True, validate = "1:1") # Reference class: 2019-01-01
# Topic_fine
reg_df = reg_df.merge(pd.get_dummies(articles_02['Topic_fine'], drop_first = True, prefix = 'topic'), 
                                how = 'inner', left_index = True, right_index = True, validate = "1:1") # Reference class: tragedies & crimes
# Source
reg_df = reg_df.merge(pd.get_dummies(articles_02['source_short'], drop_first = True, prefix = 'src'),
                                how = 'inner', left_index = True, right_index = True, validate = "1:1") # Reference class: de_NZZ


# Remove the articles_02 dataframe to save memory
del articles_02

# Transform the contained covid metrics from cases/deaths per million population to cases/deaths per 10K population
reg_df[reg_df.columns[4:8]] = reg_df[reg_df.columns[4:8]].copy() / 100
# Rename the modified columns accordingly
reg_df.rename(columns = {'weighted_dconfirmed_pm': 'weighted_dcases_p10k', 'weighted_ddeaths_pm': 'weighted_ddeaths_p10k', 
                         'weighted_7dconfirmed_pm': 'weighted_7dcases_p10k', 'weighted_7ddeaths_pm': 'weighted_7ddeaths_p10k', 'weighted_stridx': 'weighted_KOFSPI'}, inplace = True)

# Take a look at the created dataframe
reg_df

,Vader_polarity_adj,Vader_polarity_adj_2,Blob_polarity,weighted_KOFSPI,weighted_dcases_p10k,weighted_ddeaths_p10k,weighted_7dcases_p10k,weighted_7ddeaths_p10k,count,date_2019-01-02 00:00:00,...,src_fr_NNHEU,src_fr_NNTDG,src_fr_NNTLM,src_fr_NOU,src_fr_TDG,src_fr_TLMD,src_fr_TPS,src_fr_ZWAS,src_it_COOI,src_it_ZWAI
0,0.689970,0.641458,-0.161747,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.446321,-0.414940,2.062278,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.110750,-0.102963,0.363931,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.761403,-0.707869,-2.426209,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1.721049,1.600042,-0.424587,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426408,1.959519,1.821745,1.741135,61.666668,9.153322,0.401042,7.674033,0.266416,2,0,...,0,0,0,0,0,0,0,0,0,1
426409,1.282541,1.117131,1.139605,72.469512,8.094849,0.114850,7.131960,0.168062,3,0,...,0,0,0,0,0,0,0,0,0,1
426410,1.095387,0.794018,0.973308,72.469512,4.654948,0.229078,6.617441,0.192095,1,0,...,0,0,0,0,0,0,0,0,0,1
426411,1.833062,1.704180,1.628772,72.469512,8.910544,0.257012,6.808913,0.220652,2,0,...,0,0,0,0,0,0,0,0,0,1


In [22]:
## Get the feature variables to use in the regression
# Variables of interest
features = reg_df[['weighted_KOFSPI', 'weighted_dcases_p10k']].copy()
# Control variables
features = features.merge(reg_df[reg_df.columns[9:]], how = 'inner', left_index = True, right_index = True, validate = "1:1")
# Add a constant
features = sm.add_constant(features)

# Get the target variable / label
label = reg_df[['Vader_polarity_adj_2']].copy()

In [23]:
## Create the different feature selection sets to use subsequently
# Basic model without any fixed effects
selection_1 = [0,1,2]
# Add time fixed effects
selection_2 = selection_1.copy()
[selection_2.append(num) for num in np.arange(3,743)] # days
# Add cross-sectional fixed effects
selection_3 = selection_2.copy()
[selection_3.append(num) for num in np.arange(743,770)] # topics
[selection_3.append(num) for num in np.arange(770,846)] # sources

# Estimate a model using the variable of interest together with cross sectional and time fixed effects (FE)
model_04 = sm.WLS(label, features[features.columns[selection_3]], weights = reg_df['count']).fit(cov_type = 'HC3')

In [24]:
# Take a look at the results of the model
stargazer = Stargazer([model_04])
stargazer.title('Effect of the KOFSPI & Daily Confirmed Covid Cases per 10K Population on VPAII')
stargazer.custom_columns(['RC3'], [1])
stargazer.show_model_numbers(False)
stargazer.significant_digits(5)
coef_ordered = features.columns[selection_1].tolist()
stargazer.covariate_order(coef_ordered)
stargazer
#stargazer.render_latex()

# Note: Reference classes
## Topic_fine: tragedies & crimes
## Source:     de_NZZ
## Day:        2019-01-01

### Reference value for coefficient on weighted_stridx from main regression: = -0.00175
### Reference value for coefficient on weighted_dcases_p10k from main regression: = -0.00214